# Neural Style Transfer 

In [ ]:
!pip3 install tensorflow-gpu==1.15.2

     |████████████████████████████████| 411.0MB 35kB/s 
     |████████████████████████████████| 3.8MB 45.8MB/s 
     |████████████████████████████████| 512kB 42.8MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7540 sha256=da266bd9c26b629b58b1090dcdd170028c2388e1e179af511d1ef948aa74f4c3
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow 2.2.0 has requirement gast==0.3.3, but you'll have gast 0.2.2 which is incompatible.
ERROR: tensorflow 2.2.0 has requirement tensorboard<2.3.0,>=2.2.0, but you'll have tensorboard 1.15.0 which is incompatible.
ERROR: tensorflow 2.2.0 has requirement tensorflow-estimator<2.3.0,>=2.2.0, but you'll have tensorflow-estimator 1.15.1 which is incompatible.
ERROR: tensorflow-probability 0.10.0 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
  Found existing installation: tensorboard 2.2.2
    Uninstalling te

In [ ]:
!mkdir input_images
!mkdir final_images
!mv *.jpg input_images/

## Importing libraries 

In [1]:
from tensorflow.keras.applications import vgg19
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras import backend as K
import numpy as np
from scipy.optimize import fmin_l_bfgs_b
import imageio
import time

In [2]:
target_image_path = 'final_images/chittorgarh_fort_stylised_at_iteration_14.png'
style_image_path = 'input_images/style-image-1.jpg'

width, height = load_img(target_image_path).size
img_height = 600
img_width = int(width * img_height/height)

## Auxillary functions

In [3]:
def preprocess_image(image_path):
    img = load_img(image_path, target_size=(img_height, img_width))
    img = img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = vgg19.preprocess_input(img)
    return img

def deprocess_image(x):
    x[:, :, 0] += 103.939
    x[:, :, 1] += 116.779
    x[:, :, 2] += 123.68
    x = x[:, :, ::-1]
    x = np.clip(x, 0, 255).astype('uint8')
    return x 

In [4]:
target_image = K.constant(preprocess_image(target_image_path))
style_image = K.constant(preprocess_image(style_image_path))
combination_image = K.placeholder((1, img_height, img_width, 3))

input_tensor = K.concatenate([target_image, style_image, combination_image], axis=0)

model = vgg19.VGG19(input_tensor=input_tensor, weights='imagenet', include_top=False)
model.summary()

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Model: "vgg19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(3, 600, 1142, 3)]       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (3, 600, 1142, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (3, 600, 1142, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (3, 300, 571, 64)         0         
_________________________________________________________________
block2_conv1 (Conv2D)        (3, 300, 571, 128)        73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (3, 300, 571, 128)        147584    
______________________________________________

## Content Loss

In [5]:
def content_loss(base, combination):
    return K.sum(K.square(combination-base))

## Style loss (Gram Matrix)

In [6]:
def gram_matrix(x):
    features = K.batch_flatten(K.permute_dimensions(x, (2, 0, 1)))
    gram = K.dot(features, K.transpose(features))
    return gram

def style_loss(style, combination):
    S = gram_matrix(style)
    C = gram_matrix(combination)
    channels = 3
    size = img_height * img_width
    return K.sum(K.square(S - C)) / (4. * (channels ** 2) * (size ** 2))

In [7]:
def total_variation_loss(x):
    a = K.square(x[:, :img_height - 1, :img_width - 1, :] - x[:, 1:, :img_width - 1, :])
    b = K.square(x[:, :img_height - 1, :img_width - 1, :] - x[:, :img_height - 1, 1:, :])
    return K.sum(K.pow(a + b, 1.25))

## Defining final loss

In [8]:
outputs_dict = dict([(layer.name, layer.output) for layer in model.layers])
content_layer = 'block5_conv2'
style_layers = ['block1_conv1', 'block2_conv1', 'block3_conv1', 'block4_conv1', 'block5_conv1']
total_variation_weight = 1e-4
style_weight = 1.
content_weight = 0.025

loss = K.variable(0.0)
layer_features = outputs_dict[content_layer]
target_image_features = layer_features[0, :, :, :]
combination_features = layer_features[2, :, :, :]
loss = loss + content_weight * content_loss(target_image_features, combination_features)

for layer_name in style_layers:
    layer_features = outputs_dict[layer_name]
    style_features = layer_features[1, :, :, :]
    combination_features = layer_features[2, :, :, :]
    s1 = style_loss(style_features, combination_features)
    loss = loss + (style_weight / len(style_layers)) * s1

loss = loss + total_variation_weight * total_variation_loss(combination_image)

In [9]:
grads = K.gradients(loss, combination_image)[0]
fetch_loss_and_grads = K.function([combination_image], [loss, grads])

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [10]:
class Evaluator(object):
    def __init__(self):
        self.loss_value = None
        self.grad_values = None

    def loss(self, x):
        assert self.loss_value is None
        x = x.reshape((1, img_height, img_width, 3))
        outs = fetch_loss_and_grads([x])

        loss_value = outs[0]
        grad_values = outs[1].flatten().astype('float64')
        self.loss_value = loss_value
        self.grad_values = grad_values
        return self.loss_value
    
    def grads(self, x):
        assert self.loss_value is not None
        grad_values = np.copy(self.grad_values)
        self.loss_value = None
        self.grad_values = None
        return grad_values

evaluator = Evaluator()

## Style-transfer loop

In [11]:
result_prefix = 'final_images/chittorgarh_fort_stylised'
iterations = 20

x = preprocess_image(target_image_path)
x = x.flatten()
for i in range(14, iterations):
    print('Iteration number: ', i+1)
    start_time = time.time()
    x, min_val, info = fmin_l_bfgs_b(evaluator.loss, x, fprime=evaluator.grads, maxfun=20)
    print('Current loss value: ', min_val)
    img = x.copy().reshape((img_height, img_width, 3))
    img = deprocess_image(img)
    fname = result_prefix + '_at_iteration_%d.png' %(i+1)
    imageio.imwrite(fname, img)
    end_time = time.time()
    print('Iteration %d completed in %d s' %(i+1, end_time - start_time))

Iteration number:  15
Current loss value:  435653340.0
Iteration 15 completed in 1365 s
Iteration number:  16
Current loss value:  401866660.0
Iteration 16 completed in 1348 s
Iteration number:  17
Current loss value:  373471800.0
Iteration 17 completed in 1348 s
Iteration number:  18
Current loss value:  347111460.0
Iteration 18 completed in 1351 s
Iteration number:  19
Current loss value:  325017340.0
Iteration 19 completed in 1346 s
Iteration number:  20
Current loss value:  303959520.0
Iteration 20 completed in 1349 s


In [12]:
!zip -r /content/final_images.zip /content/final_images

updating: content/final_images/ (stored 0%)
  adding: content/final_images/chittorgarh_fort_stylised_at_iteration_20.png (deflated 0%)
  adding: content/final_images/chittorgarh_fort_stylised_at_iteration_19.png (deflated 0%)
  adding: content/final_images/chittorgarh_fort_stylised_at_iteration_17.png (deflated 0%)
  adding: content/final_images/chittorgarh_fort_stylised_at_iteration_15.png (deflated 0%)
  adding: content/final_images/chittorgarh_fort_stylised_at_iteration_18.png (deflated 0%)
  adding: content/final_images/chittorgarh_fort_stylised_at_iteration_16.png (deflated 0%)
  adding: content/final_images/chittorgarh_fort_stylised_at_iteration_12.png (deflated 0%)
  adding: content/final_images/chittorgarh_fort_stylised_at_iteration_14.png (deflated 0%)
  adding: content/final_images/.ipynb_checkpoints/ (stored 0%)
  adding: content/final_images/chittorgarh_fort_stylised_at_iteration_13.png (deflated 0%)


In [13]:
from google.colab import files
files.download("/content/final_images.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>